# 🎶 Social Media Tracker (v3.4 - Ultra-Robust)
**Efficient Multi-Platform Scraper & Database Sync**

### Resilience Features:
1. **Isolated Scraping**: If one platform fails, other stats are still saved.
2. **MySQL NaN Fix**: Sanitizes `NaN` to `None` for MySQL compatibility.
3. **Automatic Retries**: 3 atomic DB attempts per record.
4. **Smart Logging**: Informative warnings to identify the source of errors.

In [1]:
import os
import re
import json
import time
import requests
import pandas as pd
import numpy as np
import mysql.connector
import scrapers
from scrapers import InstagramProfile, TwitterProfile, SpotifyProfile, StubhubProfile, clean_for_mysql, convert_string_to_number

print('📦 Libraries loaded (v4.0 Modular).')


📦 Libraries loaded (v4.0 Robust).


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/var/folders/9h/ghtl82dn5bvgn2ck1qdxgf9m0000gp/T/ipykernel_74180/1365697591.py:16: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  import google.generativeai as genai


In [2]:
def load_creds(path):
    if os.path.exists(path):
        with open(path, 'r') as f: return json.load(f)
    return {}

db_creds = load_creds('postgres_credentials.json')
spotify_creds = load_creds('spotify_credentials.json')

headers = {}
if spotify_creds:
    try:
        res = requests.post('https://accounts.spotify.com/api/token', 
                            data={'grant_type': 'client_credentials', 
                                  'client_id': spotify_creds['client_id'], 
                                  'client_secret': spotify_creds['client_secret']})
        if res.status_code == 200:
            headers = {'Authorization': f'Bearer {res.json()["access_token"]}'}
            print('✅ Spotify API Authenticated.')
    except: print('⚠️ Spotify API Auth failed.')

print('🔐 Credentials configured.')

# Gemini Initialization
model = None
if HAS_GEMINI:
    try:
        gemini_creds = load_creds("gemini_credentials.json")
        if gemini_creds:
            genai.configure(api_key=gemini_creds['api_key'])
            model = genai.GenerativeModel('gemini-2.5-flash')
            print('🤖 Gemini API configured.')
    except Exception as e:
        print(f'⚠️ Gemini init failed: {e}')



def get_conn():
    # Clean credentials for mysql.connector
    creds = {k: (v.strip() if isinstance(v, str) else v) for k, v in db_creds.items()}
    if 'sslmode' in creds: del creds['sslmode']
    return mysql.connector.connect(**creds)


✅ Spotify API Authenticated.
🔐 Credentials configured.
🤖 Gemini API configured.


In [3]:
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36')

driver = webdriver.Chrome(options=chrome_options)
print('🌐 Selenium initialized.')

🌐 Selenium initialized.


In [ ]:
# Initialize modular scrapers with ready globals
scrapers.set_globals(driver, model, headers)
print('🟢 Scraper globals initialized.')


In [11]:
CACHE_HOURS = 1  # Refresh artists every X hours
conn = get_conn()
query = f'SELECT * FROM ARTISTS WHERE updated_at IS NULL OR updated_at < NOW() - INTERVAL {CACHE_HOURS} HOUR'
artists_df = pd.read_sql(query, conn)
conn.close()
print(f'📊 Loaded {len(artists_df)} artists needing updates.')


📊 Loaded 324 artists needing updates.


/var/folders/9h/ghtl82dn5bvgn2ck1qdxgf9m0000gp/T/ipykernel_74180/528903041.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  artists_df = pd.read_sql(query, conn)


In [12]:
session_summary = []
error_log = []
conn = get_conn()
print(f'🚢 Processing {len(artists_df)} artists...')

for idx, row in artists_df.iterrows():
    name = row['name']
    start_time = time.time()
    
    ig = InstagramProfile(name, clean_for_mysql(row.get('instagram_username')))
    sp = SpotifyProfile(name, clean_for_mysql(row.get('spotify_id')))
    tw = TwitterProfile(name, clean_for_mysql(row.get('twitter_username')))
    sh = StubhubProfile(name, clean_for_mysql(row.get('stubhub_url')))
    
    scraper_errors = {} # Track error message per platform
    
    for scraper, label in [(ig, 'IG'), (sp, 'Spotify'), (tw, 'Twitter'), (sh, 'Stubhub')]:
        try: 
            scraper.get_all()
        except Exception as e: 
            err_msg = str(e)
            scraper_errors[label] = err_msg
            error_log.append({'Timestamp': time.strftime('%H:%M:%S'), 'Artist': name, 'Platform': label, 'Error': err_msg})

    # --- Validation: Track which metrics actually failed (including Scraper 0 returns) ---
    failed_details = []
    metrics = [
        (ig, 'follower_count', 'instagram_followers', 'IG'),
        (tw, 'follower_count', 'twitter_followers', 'Twitter'),
        (sp, 'followers', 'spotify_followers', 'Spotify Fol'),
        (sp, 'listens', 'spotify_listeners', 'Spotify Lis'),
        (sh, 'favourites', 'stubhub_favourites', 'Stubhub')
    ]
    
    for profile, attr, db_col, label in metrics:
        scraped_val = getattr(profile, attr, 0) or 0
        if scraped_val == 0:
            curr_val = row.get(db_col, 0) or 0
            setattr(profile, attr, curr_val)
            # Use specialized error if caught, else generic
            # Extract platform name from label (e.g., 'Spotify Fol' -> 'Spotify')
            platform_key = label.split(' ')[0]
            specific_err = scraper_errors.get(platform_key, "Got 0/Null")
            failed_details.append(f"{label}: {specific_err}")
            if curr_val > 0:
                error_log.append({'Timestamp': time.strftime('%H:%M:%S'), 'Artist': name, 'Platform': label, 'Error': f'Rejected 0 update'})

    # Database Update
    success = False
    for attempt in range(3):
        try:
            q = '''
                INSERT INTO ARTISTS (
                    name, instagram_username, instagram_followers, 
                    spotify_id, spotify_genre, spotify_followers, 
                    spotify_popularity, spotify_listeners, 
                    twitter_username, twitter_followers, 
                    stubhub_url, stubhub_favourites
                ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                ON DUPLICATE KEY UPDATE
                    instagram_username=VALUES(instagram_username), 
                    instagram_followers=VALUES(instagram_followers),
                    spotify_id=VALUES(spotify_id), 
                    spotify_genre=VALUES(spotify_genre), 
                    spotify_followers=VALUES(spotify_followers),
                    spotify_popularity=VALUES(spotify_popularity),
                    spotify_listeners=VALUES(spotify_listeners),
                    twitter_username=VALUES(twitter_username), 
                    twitter_followers=VALUES(twitter_followers),
                    stubhub_url=VALUES(stubhub_url), 
                    stubhub_favourites=VALUES(stubhub_favourites),
                    updated_at=CURRENT_TIMESTAMP
            '''
            v = (name, ig.username, ig.follower_count, sp.spotifyID, sp.genre, 
                 sp.followers, sp.popularity, sp.listens, 
                 tw.username, tw.follower_count, sh.url, sh.favourites)
            v = tuple(clean_for_mysql(x) for x in v)
            with conn.cursor() as cur: cur.execute(q, v)
            success = True
            break
        except: time.sleep(2)

    if success:
        elapsed = time.time() - start_time
        fail_str = f" | ⚠️ {', '.join(failed_details)}" if failed_details else ""
        print(f'✅ {name:<25} | {elapsed:.1f}s{fail_str}')
        session_summary.append({"Artist": name, "IG": ig.follower_count, "Spotify": sp.listens, "Time": f'{elapsed:.1f}s'})
    else: print(f'❌ Final DB Failure for {name}')

conn.close()
print('🎯 Finished!')
if error_log: 
    print("\n--- Detailed Error Log ---")
    display(pd.DataFrame(error_log))
if session_summary: 
    print("\n--- Session Summary ---")
    display(pd.DataFrame(session_summary))

# Export results to files
if error_log:
    pd.DataFrame(error_log).to_csv('last_error_log.csv', index=False)
    print('📁 Saved error log to last_error_log.csv')

if session_summary:
    pd.DataFrame(session_summary).to_csv('last_session_summary.csv', index=False)
    print('📁 Saved session summary to last_session_summary.csv')


🚢 Processing 324 artists...
✅ $Uicide Boy$              | 26.4s
✅ 49Th & Main               | 62.0s | ⚠️ Stubhub: Got 0/Null
✅ 50 Cent                   | 23.9s
✅ 6Arelyhuman               | 23.6s
✅ Above And Beyond          | 51.2s
✅ Ac Slater                 | 24.9s
✅ Acraze                    | 25.9s
✅ Ado                       | 95.8s
✅ Aespa                     | 32.2s
✅ Alan Walker               | 42.2s
✅ Alex Warren               | 53.1s
✅ Alexandra Kay             | 94.2s
✅ Ali + alan                | 117.4s
✅ Alleycvt                  | 86.7s
✅ Anderson .Paak            | 101.0s
✅ Andy C                    | 102.1s
✅ Angrybaby                 | 84.7s
✅ Armin Van Buuren          | 85.2s
✅ Artemas                   | 84.9s
✅ Atarashi                  | 85.1s
✅ Atliens                   | 86.0s
✅ Audien                    | 107.7s
✅ Aurora                    | 94.4s
✅ Avril Lavigne             | 24.5s
✅ Azzeca                    | 92.3s
✅ Babbu Man                 | 100.8s
✅ Bank

,Timestamp,Artist,Platform,Error
0,16:12:38,Central Cee,Spotify Fol,Rejected 0 update
1,16:15:04,Chase Atlantic,Spotify Fol,Rejected 0 update
2,16:16:45,Chasewest,Spotify Fol,Rejected 0 update
3,16:18:10,Chelsea Cutler And Jeremy Zucker,Spotify Fol,Rejected 0 update
4,16:19:36,Chris Avant Garde,Spotify Fol,Rejected 0 update
...,...,...,...,...
685,17:18:41,Zorza,IG,Rejected 0 update
686,17:18:41,Zorza,Spotify Fol,Rejected 0 update
687,17:18:43,Zulan,IG,Rejected 0 update
688,17:18:43,Zulan,Twitter,Rejected 0 update



--- Session Summary ---


,Artist,IG,Spotify,Time
0,$Uicide Boy$,3649969.0,11300000.0,26.4s
1,49Th & Main,45602.0,955200.0,62.0s
2,50 Cent,38700813.0,46700000.0,23.9s
3,6Arelyhuman,759836.0,4000000.0,23.6s
4,Above And Beyond,668249.0,2100000.0,51.2s
...,...,...,...,...
319,Zack Fox,15000.0,458900.0,1.1s
320,Zeds Dead,47000.0,1700000.0,1.0s
321,Zhou Shen,1753.0,815200.0,1.4s
322,Zorza,18000.0,37200.0,1.6s


🛑 Browser closed.


In [ ]:
# --- SELECTIVE RETRY PASS FOR FAILURES ---
if error_log:
    # 1. Group failed platforms by artist
    retry_targets = {}
    for entry in error_log:
        artist = entry['Artist']
        # Map 'Spotify Fol' or 'Spotify Lis' back to the 'Spotify' scraper class
        platform = entry['Platform'].split(' ')[0]
        if artist not in retry_targets: retry_targets[artist] = set()
        retry_targets[artist].add(platform)

    if retry_targets:
        print(f'\n🔄 SELECTIVE RETRY PASS: Retrying specific metrics for {len(retry_targets)} artists...')
        conn = get_conn()
        
        for name, platforms in retry_targets.items():
            start_time = time.time()
            
            # Get the current row from original DF to re-init
            match_df = artists_df[artists_df['name'] == name]
            if match_df.empty: continue
            row = match_df.iloc[0].to_dict()
            
            # Initialize scrapers
            ig = InstagramProfile(name, row.get('instagram_username'))
            sp = SpotifyProfile(name, row.get('spotify_id'))
            tw = TwitterProfile(name, row.get('twitter_username'))
            sh = StubhubProfile(name, row.get('stubhub_url'))

            # Load existing known-good values into classes
            ig.follower_count = row.get('instagram_followers', 0) or 0
            sp.followers = row.get('spotify_followers', 0) or 0
            sp.popularity = row.get('spotify_popularity', 0) or 0
            sp.listens = row.get('spotify_listeners', 0) or 0
            tw.follower_count = row.get('twitter_followers', 0) or 0
            sh.favourites = row.get('stubhub_favourites', 0) or 0

            # 2. SELECTIVE SCRAPING: Only run what failed
            retry_results = []
            for p in platforms:
                try:
                    if p == 'IG': ig.get_all()
                    elif p == 'Spotify': sp.get_all()
                    elif p == 'Twitter': tw.get_all()
                    elif p == 'Stubhub': sh.get_all()
                    retry_results.append(p)
                except: pass

            # 3. Database Update
            try:
                q = '''
                    INSERT INTO ARTISTS (
                        name, instagram_username, instagram_followers, 
                        spotify_id, spotify_genre, spotify_followers, 
                        spotify_popularity, spotify_listeners, 
                        twitter_username, twitter_followers, 
                        stubhub_url, stubhub_favourites
                    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                    ON DUPLICATE KEY UPDATE
                        instagram_followers=VALUES(instagram_followers),
                        spotify_followers=VALUES(spotify_followers),
                        spotify_popularity=VALUES(spotify_popularity),
                        spotify_listeners=VALUES(spotify_listeners),
                        twitter_followers=VALUES(twitter_followers),
                        stubhub_favourites=VALUES(stubhub_favourites),
                        updated_at=CURRENT_TIMESTAMP
                '''
                v = (name, ig.username, ig.follower_count, sp.spotifyID, sp.genre, 
                     sp.followers, sp.popularity, sp.listens, 
                     tw.username, tw.follower_count, sh.url, sh.favourites)
                v = tuple(clean_for_mysql(x) for x in v)
                with conn.cursor() as cur: cur.execute(q, v)
                
                elapsed = time.time() - start_time
                print(f'✅ [RETRY] {name:<25} | {elapsed:.1f}s | Retried: {", ".join(retry_results)}')
            except Exception as e:
                print(f'❌ [RETRY] DB Failure for {name}: {e}')

        conn.close()
        print('\n🎯 Selective Retry Pass Finished!')
else:
    print('✨ No failures to retry.')



🔄 SELECTIVE RETRY PASS: Retrying specific metrics for 228 artists...
✅ [RETRY] Central Cee               | 0.6s | Retried: Spotify
✅ [RETRY] Chase Atlantic            | 0.6s | Retried: Spotify
✅ [RETRY] Chasewest                 | 0.6s | Retried: Spotify
✅ [RETRY] Chelsea Cutler And Jeremy Zucker | 0.5s | Retried: Spotify
✅ [RETRY] Chris Avant Garde         | 0.5s | Retried: Spotify
✅ [RETRY] Chris Luno                | 0.6s | Retried: Spotify
✅ [RETRY] Chyl                      | 0.5s | Retried: Spotify
✅ [RETRY] City And Colour           | 0.6s | Retried: Spotify
✅ [RETRY] Clairo                    | 0.5s | Retried: Spotify
✅ [RETRY] Claptone                  | 0.3s | Retried: Spotify
✅ [RETRY] Cloonee                   | 0.3s | Retried: Spotify
✅ [RETRY] Cochise                   | 0.4s | Retried: Spotify
✅ [RETRY] Coldplay                  | 0.6s | Retried: Spotify
✅ [RETRY] Cosmic Gate               | 0.4s | Retried: Spotify
✅ [RETRY] Counterparts              | 0.4s | Retried: S

KeyboardInterrupt: 

In [ ]:
driver.quit(); print('🛑 Browser closed.')